In [2]:
import numpy as np 

In [3]:
regions = np.loadtxt("Data/sable/sable_regions.txt")

In [4]:
regions.shape

(11805, 5)

In [5]:
indices = regions[:,0].tolist()

In [6]:
np.savetxt("Data/sable/indices.txt",indices)

In [7]:
regions[:,1:].shape

(11805, 4)

In [8]:
np.savetxt("Data/sable/regions.txt",regions[:,1:])

In [9]:
voisins = np.loadtxt("Data/sable/sable_surface.txt")

In [10]:
np.max(voisins[:,0])

11804.0

In [11]:
new_voisins = []

In [12]:
for i,j,sij in voisins:
    new_voisins.append([indices.index(i),indices.index(j),sij])

In [13]:
new_voisins = np.array(new_voisins)

In [14]:
np.savetxt("Data/sable/voisins.txt",new_voisins)

In [16]:
for i in range(512):
    plan = np.loadtxt(f"Data/sable/zplans/sable_z_{i+1}.txt")
    newplan = []
    for j in plan:
        newplan.append(indices.index(j))
    np.savetxt(f"Data/sable/new_z_plans/z{i+1}.txt",np.array(newplan))

In [17]:
indices = np.loadtxt("Data/sable/indices.txt").tolist()

In [18]:
regionsPlans =[[] for _ in range(len(indices))]

In [19]:
for i in range(512):
    plan = np.loadtxt(f"Data/sable/new_z_plans/z{i+1}.txt")
    for j in plan:
        regionsPlans[int(j)].append(i)

In [20]:
c= []

In [ ]:
for region in regionsPlans:
    c.append([region[0],region[-1]])

In [ ]:
np.savetxt("Data/sable/region_z_limits.txt",np.array(c))

In [22]:
x1  = np.loadtxt("Data/x_newplans/x1.txt",dtype=int)

In [23]:
x2  = np.loadtxt("Data/x_newplans/x2.txt",dtype=int)

In [24]:
np.union1d(x1,x2).shape

(436,)

In [25]:
np.savetxt("Data/x1_x2.txt",np.union1d(x1,x2),fmt='%i')

### Diffusion sur des données synthetiques

In [1]:
import numpy as np

In [2]:
region  = np.loadtxt("Data/regions_sans_optimization/regions.txt")

In [3]:
volume  = region[:,3]

In [4]:
np.min(volume)

2.0

In [5]:
np.max(volume)

15521.0

In [6]:
np.mean(volume)

1227.582126647936

In [7]:
np.sum(volume)/volume.shape[0]

1227.582126647936

## Test vectorization

In [1]:
import numpy as np 

In [17]:
def transform_vectorized(deltat,rho, mu, rho_m,vfom,vsom,vdom,kab):
    def Asynchronous_Transformation(volume,MB,DOM,FOM,SOM,CO2):
        x1 = MB
        x2 = DOM
        x3 = FOM
        x4 = SOM 
        x5 = CO2        
        #  we update the parameters if and only if there is microorganisms in the ball 
        # (obvious; it's microorganism that is responsible for the transformation of the organic matter)
        if x1 > 0 : 
            #first we let the microrganisms eat some from the dom in orther to grow
            if x2>0 : 
                cDOM = x2 / volume  #concentration of DOM
                temp =  vdom*cDOM*x1*deltat/(kab+cDOM) 
                if x1 >= temp  : # that the microorganisms have excess DOM 
                    x1 += temp # we let the microorganisms grow
                    x2 -= temp
                else : 
                    x1 += x2 # the microorganisms don't have enough DOM in order to grow during deltat
                    x2 = 0 # it lasts no DOM anymore in this ball
            # the decomposition of MB after dying to DOM and FOM 
            temp = mu*x1*deltat #the portion of MB to be decomposed 
            if x1 >= temp : # there is enough MB
                x1 -= temp # MB dying
                x2 += rho_m * temp #fast decomposition
                x3 += (1-rho_m) * temp #slow decomposition
            else : 
                x2 += rho_m * x1 #fast decomposition
                x3 += (1-rho_m) * x1 #slow decomposition
                x1 = 0 # it lasts no MB anymore in this ball
            # the respiration of microorganisms
            if x1 > 0 : 
                temp = rho * x1 * deltat
                if x1 >= temp : 
                    x1 -= temp 
                    x5 += temp # CO2 emission by microorganisms
                else :
                    x5 +=x1 # CO2 emission by microorganisms
                    x1 = 0
        #Transformation of SOM and FOM to DOM 
        #Transformation of SOM
        if x3 >0 :
            temp = vsom * x3 * deltat # portion of SOM that can be dissolved during deltat (SOM to DOM)
            if x3 >= temp: # there is enough SOM in the ball
                x2 += temp
                x3 -= temp
            else : 
                x2 +=x3
                x3 = 0
        #transformation of FOM 
        if x4 > 0 : 
            temp = vfom * x4 * deltat # portion of FOM that can be dissolved during deltat  (FOM to DOM)
            if x4 >= temp :  # there is enough FOM in the ball
                x2 += temp 
                x4 -= temp
            else : 
                x2 += x4 
                x4 = 0
        return x1,x2,x3,x4,x5
    return Asynchronous_Transformation


In [18]:
[rho,mu,rho_m,vfom,vsom,vdom,kab] = [float(a) for a in open("Data/boules.par").readlines()[0].split(" ")] 

In [19]:
deltat =  30/24*60*60

In [20]:
TVectorized = transform_vectorized(deltat,rho, mu, rho_m,vfom,vsom,vdom,kab)

In [21]:
vfunc = np.vectorize(TVectorized)

In [22]:
volume = np.ones((80,1))

In [23]:
[MB,DOM,FOM,SOM,CO2] = [np.zeros((80,1))]*5

In [25]:
X1,X2,X3,X4,X5 = vfunc(volume,MB,DOM,FOM,SOM,CO2)